# Tratamento dos dados

Este notebook foi utilizado para preparação dos dados para analise. Considerando que os dados normalmente encontram-se normalizados, este processo visa a desnormalização para facilitar a contagem.

In [1]:
# importando dependencias

import pandas as pd
import glob

## Carregando os dados para tratamento

Dados carregados:

- Modelo de urna
- Correspondência efetivada
- Votação

In [2]:
# carregando dados de modelo de urna

df_urna = pd.read_csv("./data/modelourna_numerointerno/modelourna_numerointerno.csv", encoding='latin1', delimiter=';', low_memory=False)

In [3]:
# carregando dados de correspondência efetivada

sec_turn_files = glob.glob("./data/ceft/ceft_2t_*.csv")
df_sec_turn = pd.concat(
    (pd.read_csv(f, encoding='latin1', delimiter=';', low_memory=False) for f in sec_turn_files), 
    ignore_index=True, 
)

In [4]:
# carregando dados de votação

df_votacao = pd.read_csv("./data/votacao_secao_2022_BR.zip", encoding='latin1', delimiter=';', compression='zip')

## Desnormalização

Neste precedimento os dados de modelo de urna, presentes somente no arquivo de modelo de urna, foram relacionados aos dados de correspondência efetivada. Os dados resultantes deste relacionamento foram posteriormente relacionado aos dados de votação pelos campos referentes à zona, seção e município de votação. 

In [5]:
# adicionando campo de modelo de urna aos dados de correspondência efetivada

find_model = lambda nr: next(model for model, min, max in df_urna.values if min <= nr <= max)
df_sec_turn['DS_MODELO_URNA'] = df_sec_turn['NR_URNA_EFETIVADA'].apply(find_model)

In [6]:
# adicionando os campos de correspondência efetivada aos dados de votação

filter = df_votacao['NR_TURNO'] == 2
df_merged = df_votacao[filter] \
    .merge(df_sec_turn, on=['NR_ZONA', 'NR_SECAO', 'CD_MUNICIPIO'], how='left', suffixes=('', '_y'))

## Testes de qualidade dos resultados

Os seguintes testes de qualidade buscam verificar se os dados resultantes dos procedimentos de preparação para analise possuem parâmetros de conformidade com os dados originais.

### Procedimentos realizados

O relacionamento entre modelo de urna e corrêspondencia efetivada foi realizado pelo campo número urna efetivada (NR_URNA_EFETIVADA) dos dados de correspondência com os campos que definem a faixa numérica para cada modelo de urna (NR_FAIXA_INICIAL, NR_FAIXA_FINAL e DS_MODELO_URNA). Assim, após este procedimento é verificado se nos registros resultantes há ocorrências de dados de corrêspondencia sem relação para dados de modelo de urna.

Logo, verificou-se que não ocorreu ausência deste relacionamento.  

In [7]:
df_sec_turn['DS_MODELO_URNA'].isna().sum()

0

Para o relacionamento entre os dados de votação e os dados resultantes do relacionamento de correspondência/modelo de urna, foram utilizados os campos de município, seção e zona ('NR_ZONA', 'NR_SECAO', 'CD_MUNICIPIO') presentes em ambos os conjuntos de dados. Após este procedimento é verificado se a quantidade de registro resultante é igual à quantidade de registros dos dados de votação.

Logo, verificou-se que os quantitativos analisados são iguais.

In [8]:
len(df_merged) == len(df_votacao[filter])


True

Ainda, neste último procedimento, é verificado se há correspondência para todos os registros do relacionamento.

Logo, verifica-se ausência de relacionamento para um registro.

In [9]:
df_merged['DS_MODELO_URNA'].isna().sum()

1

Abaixo é apresentado o registro mencionado.

In [10]:
df_merged[df_merged['DS_MODELO_URNA'].isna()].T

,1087469
DT_GERACAO,01/11/2022
HH_GERACAO,16:05:25
ANO_ELEICAO,2022
CD_TIPO_ELEICAO,2
NM_TIPO_ELEICAO,ELEIÇÃO ORDINÁRIA
NR_TURNO,2
CD_ELEICAO,545
DS_ELEICAO,ELEIÇÃO GERAL FEDERAL 2022
DT_ELEICAO,30/10/2022
TP_ABRANGENCIA,F


## Projeção

Neste procedimento e selecionado os campos de interesse para a analise e é salvo os trabalhos realizados.

In [11]:
pd.set_option('display.max_columns', None)

columns = [
    'SG_UF', 
    'SG_UE', 
    'CD_MUNICIPIO', 
    'NR_VOTAVEL', 
    'NR_LOCAL_VOTACAO', 
    'NR_ZONA', 
    'NR_SECAO',
    'NR_URNA_EFETIVADA', 
    'NM_MUNICIPIO', 
    'NM_VOTAVEL', 
    'NM_LOCAL_VOTACAO',
    'QT_VOTOS', 
    'DS_LOCAL_VOTACAO_ENDERECO', 
    'DS_MODELO_URNA', 
    'DS_DIVERGENCIA', 
    'DS_ORIGEM_VOTO'
]

df_merged[columns].head()

,SG_UF,SG_UE,CD_MUNICIPIO,NR_VOTAVEL,NR_LOCAL_VOTACAO,NR_ZONA,NR_SECAO,NR_URNA_EFETIVADA,NM_MUNICIPIO,NM_VOTAVEL,NM_LOCAL_VOTACAO,QT_VOTOS,DS_LOCAL_VOTACAO_ENDERECO,DS_MODELO_URNA,DS_DIVERGENCIA,DS_ORIGEM_VOTO
0,CE,BR,14958,13,1198,57,270,2039284.0,PACATUBA,LUIZ INÁCIO LULA DA SILVA,ESCOLA DE 1 GRAU MARIA GUIOMAR BASTOS CAVALCANTE,175,"RUA TOPAZIO, 331, VELHO TIMBO",2020.0,#NULO#,Urna Eletrônica
1,GO,BR,93734,13,1813,127,535,2159565.0,GOIÂNIA,LUIZ INÁCIO LULA DA SILVA,CENTRO DE ENSINO EM PERÍODO INTEGRAL CULTURA E...,67,"RUA LEPOLDINO DE AZEVEDO, QD. A-13",2020.0,#NULO#,Urna Eletrônica
2,SP,BR,65978,13,1198,152,205,1307043.0,JALES,LUIZ INÁCIO LULA DA SILVA,EM. PROFª JACIRA DE CARVALHO DA SILVA,95,"RUA NOVA YORK, 916",2010.0,#NULO#,Urna Eletrônica
3,SP,BR,71072,13,1252,392,226,2078886.0,SÃO PAULO,LUIZ INÁCIO LULA DA SILVA,EE. NELLO LORENZON,94,"RUA SÃO GONÇALO DO SAPUCAÍ, 61",2020.0,#NULO#,Urna Eletrônica
4,SP,BR,71072,13,1279,374,261,1337754.0,SÃO PAULO,LUIZ INÁCIO LULA DA SILVA,E M E F GAL EUCLYDES DE OLIVEIRA FIGUEIREDO,102,"RUA MARECHAL OLÍMPIO MOURÃO FILHO, 187",2010.0,#NULO#,Urna Eletrônica


In [79]:
df_merged[columns].to_parquet('data/votacao.parquet.gzip', compression='gzip')